<a href="https://colab.research.google.com/github/asilv232/DIO--Explorando-os-Recursos-de-IA-Generativa-com-Copilot-e-OpenAI/blob/main/genero.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Configuração do Ambiente no Google Colab

Primeiro, instale as dependências necessárias:

In [1]:
# Instalar as dependências
!pip install torch torchvision torchaudio
!git clone https://github.com/ultralytics/yolov5  # Clonar o repositório YOLOv5
%cd yolov5
!pip install -r requirements.txt  # Instalar requisitos do YOLOv5


fatal: destination path 'yolov5' already exists and is not an empty directory.
/content/yolov5


 Upload do Modelo para o Google Colab

Faça o upload do seu modelo customizado (bestModel.pt) para o Colab

In [3]:
from google.colab import files

# Fazer upload do arquivo bestModel.pt
uploaded = files.upload()

# O arquivo será salvo no diretório atual do Colab
model_path = list(uploaded.keys())[0]

print(f"Modelo carregado em: {model_path}")


Saving output_video(2).mp4 to output_video(2).mp4
Modelo carregado em: output_video(2).mp4


3. Upload dos Vídeos do HD para o Google Colab

Faça o upload dos vídeos diretamente do seu disco rígido:

In [ ]:
from google.colab import files

# Upload manual de vídeo do HD
uploaded = files.upload()

# Selecione o vídeo carregado
video_path = list(uploaded.keys())[0]

print(f"Vídeo carregado em: {video_path}")


4. Configuração do Modelo YOLOv5

Carregue o modelo YOLOv5 usando o arquivo de modelo que foi carregado:

In [ ]:
import torch

# Carregar o modelo YOLOv5 customizado
model = torch.hub.load('ultralytics/yolov5', 'custom', path=model_path, trust_repo=True)

# Ajustar a confiança mínima para a detecção
model.conf = 0.5


5. Processamento dos Vídeos

Processamento do vídeo para identificar e separar pessoas por gênero:

In [ ]:
import cv2
import os

# Abrir o vídeo para processamento
cap = cv2.VideoCapture(video_path)

# Definir o codec e criar o objeto de gravação do vídeo de saída
output_path = 'output_video.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, 30.0, (int(cap.get(3)), int(cap.get(4))))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Detecção de pessoas (homens e mulheres) no frame
    results = model(frame)

    # Desenhar as bounding boxes no frame
    for box in results.xyxy[0]:  # Obter as coordenadas da bounding box
        x1, y1, x2, y2, conf, cls = map(int, box[:6])
        label = f"{model.names[cls]} {conf:.2f}"
        if model.names[cls] == 'male':
            color = (255, 0, 0)  # Azul para homens
        elif model.names[cls] == 'female':
            color = (255, 192, 203)  # Rosa para mulheres
        else:
            color = (0, 255, 0)  # Verde para outras classes, se houver

        cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
        cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    # Escrever o frame processado no vídeo de saída
    out.write(frame)

cap.release()
out.release()

print(f'Processamento concluído. Vídeo de saída salvo em {output_path}')


6. Download do Vídeo Processado

Baixe o vídeo processado para o seu computador:

In [ ]:
from google.colab import files

# Baixar o vídeo processado
files.download(output_path)
